In [6]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import datetime
import joblib
import pickle
import gradio as gr
pd.set_option('display.max_columns', 300) # allow us read the full picture of dataframe
pd.set_option('display.max_rows', 300) # allow us read the full picture of dataframe

### 1. Prediction function
The prediction function loaded a normalization scaler and a classifier that were saved from the last section. 
The parameters were inputted from the interface and transformed into a test array, and the classifier was applied to the test array and output a prediction.

In [13]:
# load the normalization scaler and the model
classifier = joblib.load('X3_core_RF_classifier.pkl')
scaler = joblib.load('scaler3.gz')

def predict(decision, offer_num, eng_type, uob_alum, home_oversea, school, programme, multi_app, Institution_offer_num,
            Decision_year, Decision_month, Decision_day,
            Response_year, Response_month, Response_day
            ):
    
    Response_since_Sep01 = datetime.date(Response_year, Response_month, Response_day).toordinal() - datetime.date(2020, 9, 1).toordinal()
    Response_days = datetime.date(Decision_year, Decision_month, Decision_day).toordinal() - datetime.date(Response_year, Response_month, Response_day).toordinal()
    
    data = {'Decision_R': 1 if decision =='Reject' else 0, 
            'Response_since_Sep01': Response_since_Sep01, 
            'Decision_U': 1 if decision =='Unconditional Offer' else 0,
            'Computer Science': 1 if school =='Computer Science' else 0, 
            'uob_alum': 1 if uob_alum == 'alumni' else 0, 
            'MSc Adv Eng Man (Constr Man)PT': 1 if programme =='MSc Adv Eng Man (Constr Man)PT' else 0, 
            'offer_num': offer_num, 
            'home_oversea': 0 if home_oversea == 'Home' else 1, 
            'eng_type':eng_type,  
            'MSc Rail Sys Eng+Int w Pre FT': 1 if programme =='MSc Rail Sys Eng+Int w Pre FT' else 0,
            'MSc Phys + Tech Nuc React FT': 1 if programme =='MSc Phys + Tech Nuc React FT' else 0,
            'MSc Structural Engineering (FT)': 1 if programme =='MSc Structural Engineering (FT)' else 0,
            'Civil Engineering': 1 if school =='Civil Engineering' else 0, 
            'MSc Adv Eng Man (Constr Man)FT': 1 if programme =='MSc Adv Eng Man (Constr Man)PT' else 0,
            'Response days': Response_days,
            'MSc Advanced Chemical Eng with Bioprocessing FT': 1 if programme =='MSc Adv Eng Man (Constr Man)PT' else 0,
            'Mechanical Engineering': 1 if school =='Mechanical Engineering' else 0, 
            'Decision_C': 1 if decision =='Conditional Offer' else 0, 
            'multi_app': multi_app, 
            'Institution_offer_num': Institution_offer_num 
           }      
    
    X = pd.DataFrame(data, index=[0])
    X = scaler.fit_transform(X) 
    
    
    Y_pred = classifier.predict(X)
    if Y_pred == 0:
        return 'The applicant may reject the offer'
    else:
        return 'The applicant may accept the offer'

### 2. Test the model 
Test with the manual input value before wrap it into the interface

In [16]:
predict('Unconditional Offer', 5, 1, 'alumni','Home','School_EPS College Hub','MSc Computer Science FT', 2,5,
            2021,4,1,
           2021,4,30,
)

'The applicant may reject the offer'

### 3. Make the interface

In [17]:
demo = gr.Interface(
    fn = predict,
    inputs = [gr.Dropdown(label = 'Admission Decision', choices = ['Reject','Unconditional Offer','Conditional Offer']),
              gr.Number(label = 'Number of Offer'), 
              gr.Dropdown(label = 'English Test type, 0 for none, 1 for IELTS, 2 for TOEFL, 3 for PTE', choices = [0,1,2,3]),
              gr.Dropdown(label = 'UoB alumni or not', choices = ['alumni','not alumni']),
              gr.Dropdown(label = 'Home or Oversea', choices = ['Home','Oversea']),
              gr.Dropdown(label = 'School', choices = ['School_Business School' 'School_Chemical Engineering',
                                                     'School_Civil Engineering', 'School_Computer Science',
                                                     'School_EPS College Hub',
                                                     'School_Elec Elec and System Eng;Elec Elec and Sys Engineering',
                                                     'School_Mathematics', 'School_Mechanical Engineering',
                                                     'School_Metallurgy and Materials', 'School_Physics and Astronomy',
                                                     'School_School of Engineering;Elec Elec and Sys Engineering',
                                                     'School_Wiley']),
              gr.Dropdown(label = 'Programme', choices = ['MSc Computer Science FT', 'MSc Adv Eng Man (Constr Man)FT',
                                                       'MSc Elec + Computer Eng FT', 'MSc Artific Int & Mach Lear FT',
                                                       'MSc Human-Computer Interact FT', 'MSc Data Science FT',
                                                       'MSc Material Sci + Engineer FT', 'MSc Adv Eng Man (Projec Man)FT',
                                                       'MSc Structural Engineering (FT)', 'MSc Electric Power Systems FT',
                                                       'MSc Rail Sys Eng+Int w Pre FT', 'MSc Adv Eng Man (Generalist)FT',
                                                       'MSc Communications Engineer FT', 'MSc Advanced Mech Eng FT',
                                                       'MSc Phys + Tech Nuc React FT', 'MSc Civil Engineering PT Dubai',
                                                       'MSc Civil Engineering FT', 'MSc Advanced Chemical Eng FT',
                                                       'MSc Cyber Security FT', 'MSc Advanced Computer Sc FT',
                                                       'MSc Global Energy Technologies and Systems FT',
                                                       'MEng Comp Sci Sof En FT (Dub)', 'BSc Computer Sci FT (Dubai)',
                                                       'MSc Applied Mathematics FT', 'MSc MORSE FT', 'MSc Robotics FT',
                                                       'MSc Art Int + Comp Sc PT (Dub)', 'MSc Nuclear Decom Waste Man FT',
                                                       'PGCert Des Lear Env DL PT', 'BIA Engin Phys Sci 4Yr (Dubai)',
                                                       'BSc Art Intel and Comp Sci FT Dubai',
                                                       'MSc Adv Eng Man (Operat Man)FT',
                                                       'MSc Advanced Chemical Engineering with Energy FT',
                                                       'MSc Art Int + Comp Sc FT (Dub)',
                                                       'BIA Engineering Physical Sciences 43m Jan Dubai',
                                                       'MSc Geotechnical Eng FT', 'MSc Civ Eng and Management FT',
                                                       'MSc Computer Science PT (Dub)', 'MSc Geotechnical Eng + Mgt FT',
                                                       'MSc Industrial Project Man. PT Blended Learning',
                                                       'MSc Advanced Chemical Eng with Bioprocessing FT',
                                                       'MSc Food Safety; Hyg + Man FT', 'MSc Financial Engineering FT',
                                                       'MSc Formulation Engineering FT', 'MSc Elec Pow Syst w Adv Res FT',
                                                       'MSc Computer Science FT(Dubai)', 'MSc Road Mgt + Eng FT',
                                                       'MSc Fue Cel + Hyd Tec Blend FT', 'MSc Adv Chem Eng w Form FT',
                                                       'PGDip Adv Chem Eng with Healthcare Technology FT',
                                                       'BEng Mechanical Eng FT (Dubai)', 'MSc Rail Sys Eng + Int PT DA',
                                                       'MSc Civ Eng and Management PT', 'MSc Healthcare Technology FT',
                                                       'MSc Structural Engineering PT', 'MSc Rail Safe + Control Sys FT',
                                                       'MSc Rail Saf+Con Sys (Flex) DL', 'MSc Rail System Engin Integ DL',
                                                       'MSc Adv Eng Man (System Man)FT',
                                                       'MSc Civil Engineering Dubai (January) PT',
                                                       'MSc Advanced Mech Eng PT', 'MSc Civil Engineering PT',
                                                       'MSc Mathematical Finance FT', 'MSc Rail Sys Eng Inte DL Fixed',
                                                       'MSc Rail Systems Engin Int FT', 'MSc Civil Engineering FT Dubai',
                                                       'PGDip Art In + Com Sc FT (Dub)', 'PGDip Computer Scien FT(Dubai)',
                                                       'MSc Civil Engineering Part Time Fixed Duration',
                                                       'MSc Advanced Chemical Eng PT', 'MSc Adv Eng Man (Projec Man)PT',
                                                       'MSc Road Mgt + Eng PT', 'PGDip Rail Safe + Cont Sys FT',
                                                       'MSc Rail Saf+Con Sys DL Fixed', 'MSc Geotechnical Eng PT',
                                                       'MSc Adv Eng Man (Constr Man)PT', 'MSc Food Safety, Hyg + Man PT',
                                                       'MSc Phys + Tech Nuc React PT', 'PGCert Rail Sa+Co Sy (Flex) DL',
                                                       'MSc Advanced Chemical Engineering with Form PT',
                                                       'MSc Adv Eng Man (Generalist)PT', 'MSc Fue Cel + Hyd Tec Blend PT',
                                                       'MSc Adv Chem Eng with Healthcare Technology FT',
                                                       'MSc Rail Safe+Cont Sy w Pre FT', 'PGCert Rail Syst Eng Integ DL',
                                                       'PGCert Rail Systems Eng Int FT', 'MSc Civil Enginee FT Jan Dubai',
                                                       'MSc Adv Eng Man (Operat Man)PT', 'PGDip Civ Eng & Management FT',
                                                       'PGDip Road Mgt + Eng FT',
                                                       'MSc Geotechnical Engineering (PT; Fixed Duration)',
                                                       'PG MC Fuel Cell + Hyd Tech MC', 'MSc Communications Engineer PT',
                                                       'MSc Nuclear Decom Waste Man PT', 'PGDip Healthcare Technology FT',
                                                       'MSc Electric Power Systems PT', 'PGDip Rail Systems Eng Int FT',
                                                       'MSc Elec + Computer Eng PT', 'PGDip Civil Eng  FT (Dubai)',
                                                       'PGDip Art In + Com Sc PT (Dub)', 'PGDip Advanced Chemical Eng FT']),
              gr.Number(label = 'Multiple application number'),               
              gr.Number(label = 'Institution offer number'), 
              gr.Dropdown(label = 'Decision year', choices = [2020, 2021]),
              gr.Dropdown(label = 'Decision month', choices = list(range(1,13))),
              gr.Dropdown(label = 'Decision day', choices = list(range(1,32))),           
              gr.Dropdown(label = 'Response year', choices = [2020, 2021]),
              gr.Dropdown(label = 'Response month', choices = list(range(1,13))),
              gr.Dropdown(label = 'Response day', choices = list(range(1,32))) 
             ],           
    outputs = ["text"]
)
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://14038.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x2b43bac38b0>,
 'http://127.0.0.1:7862/',
 'https://14038.gradio.app')